In [60]:
# imports
import numpy as np
import pandas as pd

Get table of swiss cities, clean it and sort it by population.

In [61]:
cities = pd.DataFrame(pd.read_html('https://en.wikipedia.org/wiki/List_of_cities_in_Switzerland', header=1)[0])
cities.drop(cities.columns[1], axis=1, inplace=True)
cities.columns = ['Town', 'District', 'Canton', 'Town Population', 'Agglomeration Population', 'Agglomeration']
cities

,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration
0,Aarau,Aarau,AG,21506,76636.0,Aarau
1,Aarberg,Aarberg,BE,4628,NaN,-
2,Aarburg,Zofingen,AG,8197,98535.0,Olten–Zofingen
3,Adliswil,Horgen,ZH,18769,1334269.0,Zurich
4,Aesch (BL)[note 1],Arlesheim,BL,10440,541011.0,Basel (CH)
...,...,...,...,...,...,...
253,Zollikofen,Bern-Mittelland,BE,10306,410894.0,Bern
254,Zollikon,Meilen,ZH,13012,1334269.0,Zurich
255,Zug,-,ZG,30542,127095.0,Zug
256,Zürich,Zurich,ZH,415367,1334269.0,Zurich


In [62]:
cities.dtypes

Town                         object
District                     object
Canton                       object
Town Population              object
Agglomeration Population    float64
Agglomeration                object
dtype: object

Change Town Population to numeric type and sort by Town Population.

In [69]:
cities['Town Population'] = pd.to_numeric(cities['Town Population'], errors='coerce')
cities = cities[~cities['Town Population'].isnull()]
cities.sort_values('Town Population', ascending=False, inplace=True)
cities.reset_index(drop=True, inplace=True)
print(cities.dtypes)
cities.head()

Town                         object
District                     object
Canton                       object
Town Population             float64
Agglomeration Population    float64
Agglomeration                object
dtype: object


,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration
0,Zürich,Zurich,ZH,415367.0,1334269.0,Zurich
1,Geneva,-,GE,201818.0,579227.0,Genève (CH)
2,Basel,-,BS,177654.0,541011.0,Basel (CH)
3,Lausanne,Lausanne,VD,139111.0,409295.0,Lausanne
4,Bern,Bern-Mittelland,BE,133883.0,410894.0,Bern


Remove cities under with a population under 20000.

In [71]:
min_population_to_consider = 20000
cities = cities[cities['Town Population'] >= min_population_to_consider]
cities

,Town,District,Canton,Town Population,Agglomeration Population,Agglomeration
0,Zürich,Zurich,ZH,415367.0,1334269.0,Zurich
1,Geneva,-,GE,201818.0,579227.0,Genève (CH)
2,Basel,-,BS,177654.0,541011.0,Basel (CH)
3,Lausanne,Lausanne,VD,139111.0,409295.0,Lausanne
4,Bern,Bern-Mittelland,BE,133883.0,410894.0,Bern
5,Winterthur,Winterthur,ZH,111851.0,138252.0,Winterthur
6,Lucerne,Lucerne,LU,81691.0,226091.0,Lucerne
7,St. Gallen,St. Gallen,SG,75833.0,165860.0,St. Gallen
8,Lugano,Lugano,TI,63185.0,151037.0,Lugano (CH)
9,Biel/Bienne,Biel/Bienne,BE,55159.0,104542.0,Biel/Bienne
